<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: 
•Номер фактуры (InvoiceNumber)
•Дата выдачи (IssueDate)
•Общая сумма (TotalAmount).

• Методы:

o CalculateTotal(): метод для расчета общей суммы по фактуре.
o AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
o RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

public interface ILineItem
{
    string Description { get; set; }
    decimal Price { get; set; }
    int Quantity { get; set; }
    decimal Discount { get; set; }
    decimal GetTotal();
    void CalculateTotal();
}

public interface IPrintable
{
    void PrintInvoice();
}

public interface ICalculable
{
    void CalculateTotal();
}

public abstract class BaseLineItem : ILineItem, ICalculable
{
    public string Description { get; set; }
    public decimal Price { get; set; }
    public int Quantity { get; set; }
    public decimal Discount { get; set; }

    public decimal Weight { get; set; }
    public decimal ServiceDuration { get; set; }
    public decimal TaxRate { get; set; }
    public string Category { get; set; }

    protected BaseLineItem(string description, decimal price, int quantity, decimal discount = 0, decimal weight = 0, decimal serviceDuration = 0, decimal taxRate = 0, string category = "")
    {
        Description = description;
        Price = price;
        Quantity = quantity;
        Discount = discount;
        Weight = weight;
        ServiceDuration = serviceDuration;
        TaxRate = taxRate;
        Category = category;
    }

    public abstract decimal GetTotal();

    public virtual void CalculateTotal()
    {
        Console.WriteLine($"Расчёт общей суммы для: {Description}, Количество: {Quantity}, Цена: {Price}, Скидка: {Discount}");
    }

    public decimal CalculateTotalWithTax()
    {
        return (Price * Quantity - Discount) * (1 + TaxRate);
    }
}

public class LineItem : BaseLineItem
{
    public string SKU { get; set; }

    public LineItem(string description, decimal price, int quantity, decimal discount = 0, string sku = null, decimal weight = 0, decimal taxRate = 0, string category = "")
        : base(description, price, quantity, discount, weight, 0, taxRate, category)
    {
        SKU = sku;
    }

    public override decimal GetTotal()
    {
        return Price * Quantity - Discount;
    }

    public decimal GetTotal(decimal discount, decimal taxRate)
    {
        decimal totalWithDiscountAndTax = (Price * Quantity - discount) * (1 + taxRate);
        return totalWithDiscountAndTax;
    }

    public void UpdateDiscount(decimal newDiscount)
    {
        Discount = newDiscount;
        Console.WriteLine($"Скидка для товара {Description} обновлена на {newDiscount}");
    }
}

public class ServiceLineItem : LineItem
{
    public decimal ServiceFee { get; set; }
    public string ServiceType { get; set; }

    public ServiceLineItem(string description, decimal price, int quantity, decimal serviceFee, decimal discount = 0, decimal serviceDuration = 0, decimal taxRate = 0, string category = "", string serviceType = "")
        : base(description, price, quantity, discount, null, 0, taxRate, category)
    {
        ServiceFee = serviceFee;
        ServiceDuration = serviceDuration;
        ServiceType = serviceType;
    }

    public override decimal GetTotal()
    {
        return (Price + ServiceFee) * Quantity - Discount;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        Console.WriteLine($"Итог для услуги: {GetTotal()}");
    }

    public decimal CalculateTotalWithTime()
    {
        return (Price * ServiceDuration) * Quantity - Discount;
    }
}

public class ItemCollection<T> where T : ILineItem
{
    private List<T> items;

    public ItemCollection()
    {
        items = new List<T>();
    }

    public void AddItem(T item)
    {
        items.Add(item);
        Console.WriteLine($"Добавлена позиция: {item.Description}");
    }

    public void RemoveItem(T item)
    {
        items.Remove(item);
        Console.WriteLine($"Позиция удалена: {item.Description}");
    }

    public T FindItemByDescription(string description)
    {
        foreach (var item in items)
        {
            if (item.Description == description)
                return item;
        }
        return default(T);
    }

    public void DisplayAllItems()
    {
        foreach (var item in items)
        {
            Console.WriteLine($"Товар/Услуга: {item.Description}, Цена: {item.Price}, Количество: {item.Quantity}, Скидка: {item.Discount}, Общая сумма: {item.GetTotal()}");
        }
    }

    public void CalculateTotal()
    {
        decimal total = 0;
        foreach (var item in items)
        {
            total += item.GetTotal();
        }
        Console.WriteLine($"Общая сумма фактуры: {total}");
    }
}

public abstract class Invoice : IPrintable, ICalculable
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; set; }
    public string CustomerName { get; set; }
    public string PaymentMethod { get; set; }
    public string Currency { get; set; }

    protected List<ILineItem> LineItems = new List<ILineItem>();

    protected Invoice(string invoiceNumber, DateTime issueDate, string customerName, string paymentMethod, string currency)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        TotalAmount = 0;
        CustomerName = customerName;
        PaymentMethod = paymentMethod;
        Currency = currency;
    }

    public virtual void AddLine(ILineItem lineItem)
    {
        LineItems.Add(lineItem);
        CalculateTotal();
    }

    public virtual void RemoveLine(ILineItem lineItem)
    {
        LineItems.Remove(lineItem);
        CalculateTotal();
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.GetTotal();
        }
        Console.WriteLine($"Общая сумма счета: {TotalAmount}");
    }

    public void PrintInvoice()
    {
        Console.WriteLine($"Номер фактуры: {InvoiceNumber}, Дата выпуска: {IssueDate.ToShortDateString()}, " +
            $"Клиент: {CustomerName}, Способ оплаты: {PaymentMethod}, Валюта: {Currency}, " +
            $"Общая сумма: {TotalAmount}");
    }

    void ICalculable.CalculateTotal()
    {
        CalculateTotal();
    }
}


In [ ]:

public class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; }

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate, string customerName, string paymentMethod, string currency)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency)
    {
        SupplyDate = supplyDate;
    }
}
var goodsCollection = new ItemCollection<LineItem>();
LineItem goodsItem1 = new LineItem("Ноутбук", 1000, 2, 100, "SKU001", 2.5m);
LineItem goodsItem2 = new LineItem("Телефон", 500, 5, 50, "SKU002", 0.3m);

goodsCollection.AddItem(goodsItem1);
goodsCollection.AddItem(goodsItem2);

GoodsInvoice goodsInvoice = new GoodsInvoice("INV001", DateTime.Now, DateTime.Now.AddDays(7), "Иван Иванов", "Кредитная карта", "RUB");

goodsInvoice.AddLine(goodsItem1);
goodsInvoice.AddLine(goodsItem2);

goodsInvoice.PrintInvoice();


In [ ]:

public class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate, string customerName, string paymentMethod, string currency)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency)
    {
        ServiceDate = serviceDate;
    }

    public override void AddLine(ILineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Добавлена услуга: {lineItem.Description}, Дата услуги: {ServiceDate.ToShortDateString()}");
    }
}
var serviceCollection = new ItemCollection<ServiceLineItem>();
ServiceLineItem serviceItem1 = new ServiceLineItem("Консультация", 300, 1, 50, 0, 30, 0.1m, "Консультации", "Онлайн");
ServiceLineItem serviceItem2 = new ServiceLineItem("Уборка", 200, 2, 30, 0, 5, 0.2m, "Услуги", "Очистка помещений");

serviceCollection.AddItem(serviceItem1);
serviceCollection.AddItem(serviceItem2);
serviceCollection.CalculateTotal();


In [ ]:

public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed, string customerName, string paymentMethod, string currency)
        : base(invoiceNumber, issueDate, customerName, paymentMethod, currency)
    {
        ReturnAllowed = returnAllowed;
    }

    public override void CalculateTotal()
    {
        base.CalculateTotal();
        if (ReturnAllowed)
        {
            TotalAmount *= 0.9m;
            Console.WriteLine("Возврат разрешен. Применяется скидка.");
        }
    }
}
var combinedInvoice = new CombinedInvoice("INV003", DateTime.Now, true, "Сидор Сидоров", "Электронный перевод", "USD");
combinedInvoice.AddLine(goodsItem1);
combinedInvoice.AddLine(serviceItem1);
combinedInvoice.PrintInvoice();

